In [2]:
import psycopg2
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [3]:
aws_password = pd.read_csv("/Users/m33210/Desktop/Ananda's Documents/AP Projects/soccer_analytics/aws_password.csv")['password'][0]

conn = psycopg2.connect(
    host='pl-stats.c5sygygwsbg0.us-east-2.rds.amazonaws.com',
    database='postgres',
    user='postgres',
    password=aws_password,
    port='5432'
)

cur = conn.cursor()

In [4]:
# Set headers to mimic a real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; MySoccerScraper/1.0)"
}

# Step 1: Scrape the fixture list
fixture_url = "https://fbref.com/en/comps/9/2024-2025/schedule/2024-2025-Premier-League-Scores-and-Fixtures"
res = requests.get(fixture_url, headers=HEADERS)
soup = BeautifulSoup(res.content, "lxml")

# Step 2: Find the fixture table
fixtures_table = soup.find("table", {"id": "sched_2024-2025_9_1"})


In [5]:
# Step 3: Extract match links
matches = []
for row in fixtures_table.find_all("tr"):
    #finding match report link
    match_report_link = row.find("td", {"data-stat": "match_report"})
    gameweek = row.find('th', {'data-stat': 'gameweek'}).text
    if match_report_link and match_report_link.find("a"):
        match_link = match_report_link.find("a")["href"]
        match_id = match_link.split(('/'))[3]
        match_url = "https://fbref.com" + match_link
        #finding the date
        date_cell = row.find("td", {"data-stat": "date"})
        date = date_cell.text.strip() if date_cell else None
        #finding the time
        time_cell = row.find("td", {"data-stat": "start_time"})
        time = time_cell.text.strip() if time_cell else None
        #finding home team name link
        home_team_link = row.find("td", {"data-stat": "home_team"})
        home_team_name = home_team_link.text.strip()
        #finding away team name link
        away_team_link = row.find("td", {"data-stat": "away_team"})
        away_team_name = away_team_link.text.strip()
        #finding attendance
        attendance_cell = row.find("td", {"data-stat": "attendance"})
        attendance = attendance_cell.text.strip() if attendance_cell else None
        #finding referee
        referee_cell = row.find("td", {"data-stat": "referee"})
        referee = referee_cell.text.strip() if referee_cell else None

        matches.append((match_url, match_id, date, time, home_team_name, away_team_name, attendance, referee, gameweek))


In [5]:
test_tbl_qry = 'create table summary_test as select * from summary;'
cur.execute(test_tbl_qry)
conn.commit()

In [16]:
df_json = {'match_id':[], 'time':[], 'attendance':[], 'referee':[]}
#match_url, match_id, date, time, home_team_name, away_team_name, attendance, referee, gameweek
for match in matches:
    df_json['match_id'].append(match[1])
    df_json['time'].append(match[3])
    df_json['attendance'].append(match[6])
    df_json['referee'].append(match[7])
df = pd.DataFrame(df_json)


,match_id,time,attendance,referee
0,cc5b4244,20:00,"73,297",Robert Jones
1,a1d0d529,12:30,"30,014",Tim Robinson
2,34557647,15:00,"52,196",Craig Pawson
3,71618ace,15:00,"39,217",Simon Hooper
4,4efc72e4,15:00,"29,763",Michael Oliver
...,...,...,...,...
375,464cbad6,16:00,"30,263",Anthony Taylor
376,3d22336e,16:00,"27,671",Andy Madley
377,36844e73,16:00,"52,221",Tony Harrington
378,1ff370e8,16:00,"11,238",Lewis Smith


In [17]:
columns = ", ".join(
    [f'"{col}" TEXT' for col in df.columns]  # Default all to TEXT; you can customize types
)
table_name = "additional_data"
create_table_sql = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns});'
cur.execute(create_table_sql)
conn.commit()
for row in df.itertuples(index=False):
    values = tuple(row)
    placeholders = ', '.join(['%s'] * len(values))
    insert_sql = f'INSERT INTO {table_name} VALUES ({placeholders})'
    cur.execute(insert_sql, values)
    conn.commit()
print(f"Table `{table_name}` created and data inserted.")

Table `additional_data` created and data inserted.


In [6]:
table_name = ["summary", "passing", "defense", "passing_types", "possession", "misc"]
for tbl in table_name:
    qry = f'alter table {tbl} add column time text, add column attendance text, add column referee text;'
    cur.execute(qry)
    conn.commit()
    qry = f'update {tbl} a set time = b.time, attendance = b.attendance, referee = b.referee from additional_data b where a.match_id = b.match_id;'
    cur.execute(qry)
    conn.commit()

In [34]:
conn.rollback()

In [21]:
qry = f'select * from additional_data limit 10;'
cur.execute(qry)
print(cur.fetchall())

[('cc5b4244', '20:00', '73,297', 'Robert Jones'), ('a1d0d529', '12:30', '30,014', 'Tim Robinson'), ('34557647', '15:00', '52,196', 'Craig Pawson'), ('71618ace', '15:00', '39,217', 'Simon Hooper'), ('4efc72e4', '15:00', '29,763', 'Michael Oliver'), ('c0e3342a', '15:00', '60,261', 'Jarred Gillett'), ('eac7c00b', '17:30', '62,463', 'Tony Harrington'), ('b63822b9', '14:00', '16,988', 'Samuel Barrott'), ('67a0c715', '16:30', '39,818', 'Anthony Taylor'), ('62eea1d6', '20:00', '31,977', 'Chris Kavanagh')]


In [8]:
qry = f'select * from passing limit 10;'
cur.execute(qry)
print(cur.fetchall())

[('Bruno Fernandes', '8.0', 'pt POR', 'FW,AM', '29-343', '90', '43', '60', '71.7', '693', '229', '21', '22', '95.5', '14', '19', '73.7', '4', '9', '44.4', '0', '0.0', '0.0', '0', '4', '0', '0', '6', '2024-08-16', 'cc5b4244', '1', 'Manchester Utd', 'Fulham', 'home', '507c7bdf', '20:00', '73,297', 'Robert Jones'), ('Marcus Rashford', '10.0', 'eng ENG', 'LW', '26-290', '90', '22', '30', '73.3', '240', '92', '15', '17', '88.2', '2', '8', '25.0', '1', '1', '100.0', '0', '0.7', '0.1', '1', '2', '1', '0', '2', '2024-08-16', 'cc5b4244', '1', 'Manchester Utd', 'Fulham', 'home', 'a1d5bd30', '20:00', '73,297', 'Robert Jones'), ('Amad Diallo', '16.0', 'ci CIV', 'RW', '22-036', '60', '28', '33', '84.8', '312', '66', '17', '17', '100.0', '6', '7', '85.7', '1', '3', '33.3', '0', '0.1', '0.1', '1', '1', '0', '0', '1', '2024-08-16', 'cc5b4244', '1', 'Manchester Utd', 'Fulham', 'home', '9dc96f10', '20:00', '73,297', 'Robert Jones'), ('Alejandro Garnacho', '17.0', 'ar ARG', 'RW', '20-046', '30', '10', '1

In [35]:
qry = f"select column_name from information_schema.columns where table_name = 'summary_test';"
cur.execute(qry)
print(cur.fetchall())

[('Player',), ('#',), ('Nation',), ('Pos',), ('Age',), ('Min',), ('Gls',), ('Ast',), ('PK',), ('PKatt',), ('Sh',), ('SoT',), ('CrdY',), ('CrdR',), ('Touches',), ('Tkl',), ('Int',), ('Blocks',), ('xG',), ('npxG',), ('xAG',), ('SCA',), ('GCA',), ('Passes_cmp',), ('Passes_att',), ('Passes_cmp_pct',), ('PrgP',), ('Carries',), ('PrgC',), ('Take-ons_att',), ('Take-ons_succ',), ('game_date',), ('match_id',), ('gameweek',), ('team',), ('opponent',), ('location',), ('player_id',), ('time',), ('attendance',), ('referee',)]
